In [2]:
import json
import pickle
import numpy as np
import nltk
from tqdm.auto import tqdm

In [3]:
def unpack(file):
    with open(file, "r", encoding='utf-8') as f:
        json_list = list(f)

    all_news_text_X = []
    all_news_summ_Y = []
    for i, str in enumerate(json_list):
        one_news = json.loads(str)
        del one_news['url']
        del one_news['title']
        del one_news['date']
        all_news_text_X.append(one_news['text'])
        all_news_summ_Y.append(one_news['summary'])
    return all_news_text_X, all_news_summ_Y

In [4]:
def score(vec_X, vec_Y):
    # count angle between center vectors
    vec_X_unit = (vec_X / np.linalg.norm(vec_X)) if np.linalg.norm(vec_X) != 0 else np.zeros(vec_X.shape)
    vec_Y_unit = (vec_Y / np.linalg.norm(vec_Y)) if np.linalg.norm(vec_Y) != 0 else np.zeros(vec_Y.shape)
    angles = np.arccos(np.clip(np.dot(vec_X_unit, vec_Y_unit), -1.0, 1.0))
    # привожу [0 - хорошо, pi/2 - плохо] к [0 - плохо, 1 - хорошо]
    score_01 = 1 - (angles / (np.pi/2))
    return score_01

In [ ]:
'''
по предсказанным индексам доставать предложения из Х
посчитать s для каждого с каждым из Х и Y (для 1 текста) (score - доля совпадения от 0 до 1)
для каждого предложения Y взять предложение Х с максимальным score (каждое предложение только 1 раз)
итоговая оценка метода - среднее от суммы выбранных score
'''

In [5]:
def calc_best_scores_texts(index_sent_for_summ):
    all_best_scores_text = [] # средний score по текстам
    
    for text_i in tqdm(range(len(X_tfidf_texts_test))):
        sent_X = X_tfidf_texts_test[text_i].toarray()
        sent_for_summ_predict = sent_X[index_sent_for_summ[text_i]]
        sent_for_summ_real = Y_tfidf_texts_test[text_i].toarray()
    
        best_scores_sent_this_text = []
        can_be_choosed_Y = np.zeros(len(sent_for_summ_real), dtype=bool)  # маска индексов Y , кот еще участвуют
        for vec_x in sent_for_summ_predict:
            all_scores = np.ma.array([score(vec_x, vec_y) for vec_y in sent_for_summ_real], mask=can_be_choosed_Y)
            best_scores_sent_this_text.append(np.max(all_scores))
            can_be_choosed_Y[np.argmax(all_scores)] = True

        all_best_scores_text.append(np.sum(best_scores_sent_this_text)/len(best_scores_sent_this_text))
            
    return all_best_scores_text

### Main

In [4]:
with open('tree_index_sent_for_summ_X_test.pickle', 'rb') as f:
    index_sent_for_summ_tree = pickle.load(f)
with open('cluster_index_sent_for_summ_X_test.pickle', 'rb') as f:
    index_sent_for_summ_cluster = pickle.load(f)
with open('luhn_index_sent_for_summ_X_test.pickle', 'rb') as f:
    index_sent_for_summ_luhn = pickle.load(f)
with open('X_tfidf_texts_test.pickle', 'rb') as f:
    X_tfidf_texts_test = pickle.load(f)
with open('Y_tfidf_texts_test.pickle', 'rb') as f:
    Y_tfidf_texts_test = pickle.load(f)

In [5]:
index_sent_for_summ_tree = np.array(index_sent_for_summ_tree)
index_sent_for_summ_cluster = np.array(index_sent_for_summ_cluster)
index_sent_for_summ_luhn = np.array(index_sent_for_summ_luhn)

In [6]:
all_best_scores_text_tree = calc_best_scores_texts(index_sent_for_summ_tree)
all_best_scores_text_cluster = calc_best_scores_texts(index_sent_for_summ_cluster)
all_best_scores_text_luhn = calc_best_scores_texts(index_sent_for_summ_luhn)

[0.0256729249130353, 0.23405374259176293] 0.1298633337523991
[0.014928072858538477, 0.04887725878243421, 0.028656235827061782, 0.0] 0.023115391867008617
[0.09169394370732253, 0.20149708639183228, 0.0] 0.09773034336638493
[0.1730294088288037, 0.06077029450631022, 0.22036702911778272] 0.1513889108176322
[0.06442388652740794, 0.045408794964422405, 0.03253794465899407] 0.047456875383608134
[0.21689010281838395, 0.0, 0.2668812218317862, 0.11035106242014092] 0.14853059676757777
[0.061023845310278935, 0.0, 0.040427320830862645] 0.033817055380380524
[0.0963364625340708, 0.03862140205980946, 0.0] 0.044985954864626754
[0.08682707525142686, 0.12875702987823645, 0.0, 0.0] 0.053896026282415826
[0.08128701129188898, 0.0, 0.015187717885166818] 0.0321582430590186
[0.0, 0.015232688131658678] 0.007616344065829339
[0.023029613449763953] 0.023029613449763953
[0.13243929972869173, 0.0] 0.06621964986434586
[0.037097404407835044, 0.04887725878243421, 0.0, 0.0] 0.021493665797567313
[0.0, 0.20149708639183228, 

In [8]:
with open('all_best_scores_text_tree.pickle', 'wb') as f:
    pickle.dump(all_best_scores_text_tree, f)
with open('all_best_scores_text_cluster.pickle', 'wb') as f:
    pickle.dump(all_best_scores_text_cluster, f)
with open('all_best_scores_text_luhn.pickle', 'wb') as f:
    pickle.dump(all_best_scores_text_luhn, f)

In [16]:
print(np.sum(all_best_scores_text_tree)/len(all_best_scores_text_tree))
print(np.sum(all_best_scores_text_cluster)/len(all_best_scores_text_cluster))
print(np.sum(all_best_scores_text_luhn)/len(all_best_scores_text_luhn))

0.08870041045497278
0.09278036692290285
0.09100806481352615


In [19]:
print(np.max(all_best_scores_text_tree))
print(np.max(all_best_scores_text_cluster))
print(np.max(all_best_scores_text_luhn))

print(np.argmax(all_best_scores_text_tree))
print(np.argmax(all_best_scores_text_cluster))
print(np.argmax(all_best_scores_text_luhn))

0.6172737639590911
0.7771379789725386
0.5656863302557644
1411
582
3965


### Получение текста и его аннотации 

In [6]:
all_news_text_X, all_news_summ_Y = unpack(
    'C:/Users/Acer/PycharmProjects/ML_NLP_course3/gazeta_jsonl/gazeta_test.jsonl'
)

In [9]:
with open('tree_index_sent_for_summ_X_test.pickle', 'rb') as f:
    index_sent_for_summ_tree = pickle.load(f)
with open('cluster_index_sent_for_summ_X_test.pickle', 'rb') as f:
    index_sent_for_summ_cluster = pickle.load(f)
with open('luhn_index_sent_for_summ_X_test.pickle', 'rb') as f:
    index_sent_for_summ_luhn = pickle.load(f)

In [21]:
with open('all_best_scores_text_tree.pickle', 'rb') as f:
    all_best_scores_text_tree = pickle.load(f)
with open('all_best_scores_text_cluster.pickle', 'rb') as f:
    all_best_scores_text_cluster = pickle.load(f)
with open('all_best_scores_text_luhn.pickle', 'rb') as f:
    all_best_scores_text_luhn = pickle.load(f)

In [43]:
print(np.argmin([a for a in all_best_scores_text_cluster[45:]]))

3


In [63]:
print(np.argmin(np.array([len(nltk.sent_tokenize(text)) for text in all_news_text_X])))
print(np.min(np.array([len(nltk.sent_tokenize(text)) for text in all_news_text_X])))

1851
13


In [44]:
text_index = 48
print(all_best_scores_text_cluster[text_index])
text_X_indexes_summ = np.sort(index_sent_for_summ_cluster[text_index])
print(text_X_indexes_summ)
text_X = all_news_text_X[text_index]
text_X_token = nltk.sent_tokenize(text_X)
text_Y = all_news_summ_Y[text_index]

0.0
[10 13]


In [45]:
print("Исходный текст")
print(text_X + '\n')
print("Оригинальная аннотация")
print(all_news_summ_Y[text_index] + '\n')
print("Сгенерированная аннотация")
print(" ".join([text_X_token[index_summ] for index_summ in text_X_indexes_summ]))

Исходный текст
Ведущий популярного телешоу «Караоке в машине» Джеймс Корден прокомментировал недавний скандал со своим участием, когда пользователи интернета уличили его в том, что в действительности не он водит автомобиль во время съемок своей передачи. Видеообращение Кордена было опубликовано на YouTube-канале «The Late Late Show with James Corden». Телеведущий начал с того, что назвал неправдой все обвинения в свой адрес. Далее он продемонстрировал видео, которое попало несколько дней назад в Twitter. На нем видно, что Корден и поп-звезда Джастин Бибер едут в машине, на которой установлен специальный прицеп, и ее везет грузовик. «Я знаю — это выглядит плохо, — сказал знаменитый англичанин, выдержав драматическую паузу. — Но я просто хочу сказать сейчас, что я всегда сам вожу машину, пока мы не делаем что-то, что может показаться небезопасным. Например, танцы, смена костюмов или если я пьян». Что касается ситуации с Бибером , по словам ведущего, решение воспользоваться страховкой был

In [12]:
i = 0
while i<50:
    print('\n')
    i+=1